In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/daniel-dc-cd/data_science/master/module_4_ML/data/seattle_weather_1948-2017.csv')

numrows = 25549 

In [3]:
df.head()

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True


In [4]:

regression = pd.DataFrame({'intercept':[1] * numrows,
                           'yesterday':[0.0] * numrows,
                             'today':[0.0] * numrows,
                             'tomorrow':[0.0] * numrows
                          })

In [5]:
seq = ['intercept',
       'yesterday',
       'today',
        'tomorrow']
regression = regression.reindex(columns=seq)

In [6]:
regression.head()

,intercept,yesterday,today,tomorrow
0,1,0.0,0.0,0.0
1,1,0.0,0.0,0.0
2,1,0.0,0.0,0.0
3,1,0.0,0.0,0.0
4,1,0.0,0.0,0.0


In [7]:
for i in range(0 , numrows):
    tomorrow = df.iloc[i,1]
    today = df.iloc[(i-1),1]
    yesterday= df.iloc[(i-2),1]
    regression.iat[i,3] = tomorrow
    regression.iat[i,2] = today
    regression.iat[i,1] = yesterday

In [8]:
regression.head()

,intercept,yesterday,today,tomorrow
0,1,0.00,0.00,0.47
1,1,0.00,0.47,0.59
2,1,0.47,0.59,0.42
3,1,0.59,0.42,0.31
4,1,0.42,0.31,0.17


In [9]:
regression.describe

<bound method NDFrame.describe of        intercept  yesterday  today  tomorrow
0              1       0.00   0.00      0.47
1              1       0.00   0.47      0.59
2              1       0.47   0.59      0.42
3              1       0.59   0.42      0.31
4              1       0.42   0.31      0.17
...          ...        ...    ...       ...
25544          1       0.00   0.00      0.00
25545          1       0.00   0.00      0.00
25546          1       0.00   0.00      0.00
25547          1       0.00   0.00      0.00
25548          1       0.00   0.00      0.00

[25549 rows x 4 columns]>

In [10]:
sum_pf_tday_yday=(regression['yesterday']+regression['today'])

In [11]:
avg=sum_pf_tday_yday/numrows
avg

0        0.000000
1        0.000018
2        0.000041
3        0.000040
4        0.000029
           ...   
25544    0.000000
25545    0.000000
25546    0.000000
25547    0.000000
25548    0.000000
Length: 25549, dtype: float64

In [12]:
regression_updated=pd.DataFrame({'intercept': []*numrows
                                })
regression_updated.head()

,intercept


In [13]:
regression_updated['yesterday_today'] =avg
regression_updated.head()

,intercept,yesterday_today
0,NaN,0.000000
1,NaN,0.000018
2,NaN,0.000041
3,NaN,0.000040
4,NaN,0.000029


In [15]:
regression_updated['intercept']=regression['intercept']
regression_updated['tomorrow']=regression['tomorrow']
regression_updated.head()

,intercept,yesterday_today,tomorrow
0,1,0.000000,0.47
1,1,0.000018,0.59
2,1,0.000041,0.42
3,1,0.000040,0.31
4,1,0.000029,0.17


In [16]:
regression_updated.describe

<bound method NDFrame.describe of        intercept  yesterday_today  tomorrow
0              1         0.000000      0.47
1              1         0.000018      0.59
2              1         0.000041      0.42
3              1         0.000040      0.31
4              1         0.000029      0.17
...          ...              ...       ...
25544          1         0.000000      0.00
25545          1         0.000000      0.00
25546          1         0.000000      0.00
25547          1         0.000000      0.00
25548          1         0.000000      0.00

[25549 rows x 3 columns]>

In [17]:
print(regression_updated.iloc[0:200:,0:2])

     intercept  yesterday_today
0            1         0.000000
1            1         0.000018
2            1         0.000041
3            1         0.000040
4            1         0.000029
..         ...              ...
195          1         0.000000
196          1         0.000000
197          1         0.000000
198          1         0.000000
199          1         0.000000

[200 rows x 2 columns]


In [18]:
print(regression_updated.iloc[0:200:,2])

0      0.47
1      0.59
2      0.42
3      0.31
4      0.17
       ... 
195    0.00
196    0.00
197    0.00
198    0.00
199    0.05
Name: tomorrow, Length: 200, dtype: float64


In [19]:
def gradientDescent(X, y ,param, alpha, num_iters):
    """
       Performs gradient descent to learn param
    """
    for i in range(num_iters):
        y_hat = np.dot(X , param)
    
        param = param - alpha * np.dot(X.T, y_hat-y)
    return param

In [20]:
X = regression.iloc[0:200:,0:2] #intercept and yesterday >> here I tried to add today  to be 0:3 but  its occur an shape Error beacuse of the dot ...
y = regression.iloc[0:200:,2]   # tomrrow 
param = np.array([3,3])
alpha = 0.0001 
num_iters = 1000

solution = gradientDescent(X, y,  param, alpha, num_iters)
solution

array([0.01355702, 0.84476982])

In [21]:
regression_updated =regression_updated[~regression_updated.isin([np.nan, np.inf, -np.inf]).any(1)] # I did this code cause Its occur an Error in the "mymodel" cell 
#ValueError: Input contains NaN, infinity or a value too large for dtype('float64)

In [22]:
from sklearn import linear_model

x = regression_updated.yesterday_today.values
y = regression_updated.tomorrow.values
x = x.reshape(regression_updated.shape[0], 1)
y = y.reshape(regression_updated.shape[0], 1)

In [25]:
model = linear_model.LinearRegression().fit(x,y)

In [24]:
from sklearn.metrics import r2_score
r2_score(y,mymodel.predict(x))

0.08228912443411895

In [45]:

numrows = 25547 

regression_updated = pd.DataFrame({'yesterday_today':[0.0]*numrows,
                             'tomorrow':[0.0]*numrows})

#sort columns for convience
seq = ['yesterday_today',
       'tomorrow']

regression_updated = regression_updated.reindex(columns=seq)

for i in range(0 , numrows):
    tomorrow = df.iloc[i,1]
    yesterday_today = df.iloc[(i-1),1]
    regression_updated.iat[i,1] = tomorrow
    regression_updated.iat[i,0] = yesterday_today

regression_updated = regression_updated.dropna()

In [46]:
regression_updated.head()

,yesterday_today,tomorrow
0,0.00,0.47
1,0.47,0.59
2,0.59,0.42
3,0.42,0.31
4,0.31,0.17
